In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
# ModuleNotFoundError: No module named 'tensorflow.keras.layers.advanced_activations'
# => from tensorflow.keras.layers import LeakyReLU

In [2]:
def load_data():
    (X_train, y_train), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32)-127.5)/127.5
    X_train = X_train.reshape(60000, -1)
    return (X_train, y_train)
    
x_train, y_train = load_data()
print(x_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [3]:
def build_generator():
    model = Sequential()
    model.add(Dense(units=128, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=784, activation='tanh'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001, beta_1=0.9))
    return model

generator = build_generator()
print(generator.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12928     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 784)               4

In [4]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(units=512, input_dim=784))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Dense(units=256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Dense(units=128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001, beta_1=0.9))
    return model

discriminator = build_discriminator()
print(discriminator.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [5]:
def build_GAN(dis, gen):
    dis.trainable = False
    GAN_input = Input(shape=(100, ))
    GAN_output = dis(gen(GAN_input))
    GAN = Model(inputs=GAN_input, outputs=GAN_output)
    GAN.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.002, beta_1=0.5))
    return GAN

GAN = build_GAN(dis=discriminator, gen=generator)
print(GAN.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               579728    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 566273    
                                                                 
Total params: 1,146,001
Trainable params: 579,728
Non-trainable params: 566,273
_________________________________________________________________
None


In [10]:
def draw_images(gen, epoch, examples=25, dim=(5, 5), figsize=(10, 10)):
    noise = np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_imgs = generator.predict(noise)
    generated_imgs = generated_imgs.reshape(25, 28, 28)
    plt.figure(figsize=figsize)
    for i in range(generated_imgs.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_imgs[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('generated_images_epoch_{}'.format(epoch))
    
    
def train_GAN(epoch=1, batch_size=128):
    # ----- datasets -----
    x_train, y_train = load_data()
    # ----- GAN model -----
    generator = build_generator()
    discriminator = build_discriminator()
    GAN = build_GAN(dis=discriminator, gen=generator)
    # ----- training -----
    for i in range(1, epoch+1):
        print("===== Epoch {} =====".format(i))
        for _ in tqdm(range(batch_size)):
            noise = np.random.normal(0, 1, (batch_size, 100))
            fake_imgs = generator.predict(noise)
            real_imgs = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
            fake_label = np.zeros(batch_size)
            real_label = np.ones(batch_size)
            x = np.concatenate([fake_imgs, real_imgs])
            y = np.concatenate([fake_label, real_label])
            
            discriminator.trainable = True
            discriminator.train_on_batch(x, y)
            
            discriminator.trainable = False
            GAN.train_on_batch(noise, real_label)
            
            if i==1 or (not i%2):
                draw_images(generator, i)

In [ ]:
train_GAN(epoch=10, batch_size=128)

===== Epoch 1 =====


  0%|                                                                                          | 0/128 [00:00<?, ?it/s]

1/1 [==============================] - 0s 23ms/step


  1%|▋                                                                                 | 1/128 [00:04<08:39,  4.09s/it]

1/1 [==============================] - 0s 22ms/step


...

1/1 [==============================] - 0s 22ms/step


 49%|███████████████████████████████████████▊                                         | 63/128 [01:21<00:49,  1.30it/s]

1/1 [==============================] - 0s 21ms/step


 50%|███████████████████████████████████████                                       | 64/128 [13:32<3:54:24, 219.75s/it]

1/1 [==============================] - 0s 296ms/step
